In [ ]:
#----Patient ETL
import psycopg2
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
#from tqdm import tqdm
from datetime import datetime, timezone
#import datetime  # module
#from datetime import datetime as dt  # class, aliased to avoid conflict
#from datetime import datetime as tz
#import datetime
import json

# Path to your service account JSON key
key_path = "/Users/toniventura/keys/bq_key.json" 

# Create credentials and BigQuery client
#credentials = service_account.Credentials.from_service_account_file(key_path)
credentials = service_account.Credentials.from_service_account_file(key_path)

# Postgres config
PG_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "fhir",
    "user": "toniventura",
    "password": "fhir_project"
}
#records =[]

# BigQuery config
#BQ_PROJECT = "your-gcp-project"
BQ_PROJECT = "fhir-synthea-data"
BQ_DATASET = "fhir_curated"
#client = bigquery.Client(project=BQ_PROJECT)
client = bigquery.Client(project="fhir-synthea-data", credentials=credentials)
#client = bigquery.Client(project=BQ_PROJECT, credentials=credentials)
dataset_ref = bigquery.Dataset(f"{BQ_PROJECT}.{BQ_DATASET}")

# Helper: fetch staged data
def fetch_staged_data(table, batch_size=10000):
    try:
        conn = psycopg2.connect(**PG_CONFIG)
        cur = conn.cursor()
        #cur.execute(f"SELECT * FROM fhir_staging.{table}")
        cur.execute("SELECT * FROM fhir_staging.patients_fhir_raw LIMIT 5;")
        while True:
            rows = cur.fetchmany(batch_size)
            print(f"rows: {rows}")
            if not rows:
                break
            yield rows
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Postgres connection or query failed: {e}")
        


# Helper: insert dataframe into BigQuery
def insert_to_bq(df, table_name):
    table_id = f"{BQ_PROJECT}.{BQ_DATASET}.{table_name}"
    job = client.load_table_from_dataframe(df, table_id)
    job.result()  # wait for completion

# Example: Transform & load Patients
def transform_patients(rows):
    #print("transforming data")
    records = []
    print(f"rows: {len(rows)}")
    for r in rows:
        print("inside loop")
        rid, resource = r[1], r[2] # adjust index if needed
        #print(f"rid: {rid}")
        #print(f"resource: {resource}")
        #birth_date = resource.get("birthdate")
        records.append({
            "patient_id": rid,
            "first_name": resource.get("name", [{}])[0].get("given", [""])[0],
            "last_name": resource.get("name", [{}])[0].get("family", ""),
            #"birth_date": datetime.date.fromisoformat(resource.get("birthDate"))
                #if resource.get("birthDate") else None,
            "birth_date": datetime.fromisoformat(resource.get("birthDate"))
                if resource.get("birthDate") else None,
            "gender": resource.get("gender"),
            #"load_timestamp" : datetime.datetime.utcnow()
            "load_timestamp": datetime.now(timezone.utc)
        })
    return pd.DataFrame(records)

# Main ETL loop
def etl_patients():
    try:
        for batch in tqdm(fetch_staged_data("patients_fhir_raw")):
            df = transform_patients(batch)
            if not df.empty:
                insert_to_bq(df, "patients")
                print("***Inserting***")
    except Exception as e:
        print(f"Error in patients ETL: {e}")

if __name__ == "__main__":
    etl_patients()




In [ ]:
#---Practitioner ETL
import psycopg2
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from datetime import datetime, timezone
import json
import pprint

# Path to your service account JSON key
key_path = "/Users/toniventura/keys/bq_key.json"

# Create credentials and BigQuery client
credentials = service_account.Credentials.from_service_account_file(key_path)
BQ_PROJECT = "fhir-synthea-data"
BQ_DATASET = "fhir_curated"
client = bigquery.Client(project=BQ_PROJECT, credentials=credentials)

# Postgres config
PG_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "fhir",
    "user": "toniventura",
    "password": "fhir_project"
}

# Helper: fetch staged data
def fetch_staged_data(table, batch_size=10000):
    try:
        conn = psycopg2.connect(**PG_CONFIG)
        cur = conn.cursor()
        cur.execute(f"SELECT * FROM fhir_staging.{table} LIMIT 5;")
        while True:
            rows = cur.fetchmany(batch_size)
            if not rows:
                break
            yield rows
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Postgres connection or query failed: {e}")

# Helper: insert dataframe into BigQuery
def insert_to_bq(df, table_name):
    table_id = f"{BQ_PROJECT}.{BQ_DATASET}.{table_name}"
    job = client.load_table_from_dataframe(df, table_id)
    job.result()

# Transform Practitioners
def transform_practitioners(rows):
    records = []
    for r in rows:
        rid, resource = r[1], r[2]  # adjust index if needed

        #Initialize variables so they exist even if not found
        npi = None
        license_number = None
        other_ids = []

    for ident in resource.get("identifier", []):
        # Debugging: show the whole identifier object
        pprint.pprint(ident)

        system = ident.get("system")
        value = ident.get("value")

        print(f"system: {system}")
        print(f"value: {value}")

        if system == "http://hl7.org/fhir/sid/us-npi":
            npi = value
        elif system == "http://example.org/license-number":
            license_number = value
        else:
            other_ids.append(value)


        print("---- Results ----")
        print(f"NPI: {npi}")
        print(f"License Number: {license_number}")
        print(f"Other IDs: {other_ids}")
        name_info = resource.get("name", [{}])[0]

        records.append({
            "practitioner_id": rid,
            "first_name": name_info.get("given", [""])[0],
            "last_name": name_info.get("family", ""),
            "prefix": name_info.get("prefix", [""])[0] if name_info.get("prefix") else None,
            "gender": resource.get("gender"),
            #"birth_date": datetime.fromisoformat(resource.get("birthDate")) if resource.get("birthDate") else None,
            "npi": npi,
            "license_number": license_number,
            "primary_email": next((t.get("value") for t in resource.get("telecom", []) if t.get("system") == "email"), None),
            "primary_phone": next((t.get("value") for t in resource.get("telecom", []) if t.get("system") == "phone"), None),
            "load_timestamp": datetime.now(timezone.utc)
        })
    return pd.DataFrame(records)

# Main ETL loop
def etl_practitioners():
    try:
        for batch in fetch_staged_data("practitioners_fhir_raw"):
            df = transform_practitioners(batch)
            if not df.empty:
                insert_to_bq(df, "practitioners")
                print("***Inserted batch***")
    except Exception as e:
        print(f"Error in practitioners ETL: {e}")

if __name__ == "__main__":
    etl_practitioners()


{'system': 'http://hl7.org/fhir/sid/us-npi', 'value': '9999928192'}
system: http://hl7.org/fhir/sid/us-npi
value: 9999928192
---- Results ----
NPI: 9999928192
License Number: None
Other IDs: []
***Inserted batch***


In [ ]:
#---Practitioner Roles ETL
import psycopg2
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from datetime import datetime, timezone
import json
import pprint

# Path to your service account JSON key
key_path = "/Users/toniventura/keys/bq_key.json"

# Create credentials and BigQuery client
credentials = service_account.Credentials.from_service_account_file(key_path)
BQ_PROJECT = "fhir-synthea-data"
BQ_DATASET = "fhir_curated"
client = bigquery.Client(project=BQ_PROJECT, credentials=credentials)

# Postgres config
PG_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "fhir",
    "user": "toniventura",
    "password": "fhir_project"
}

def insert_to_bq(df, table_name):
    table_id = f"{BQ_PROJECT}.{BQ_DATASET}.{table_name}"
    job = client.load_table_from_dataframe(df, table_id)
    job.result()

def fetch_staged_data(table, batch_size=10000):
    try:
        conn = psycopg2.connect(**PG_CONFIG)
        cur = conn.cursor()
        cur.execute(f"SELECT * FROM fhir_staging.{table} LIMIT 5;")
        while True:
            rows = cur.fetchmany(batch_size)
            if not rows:
                break
            yield rows
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Postgres connection or query failed: {e}")

# Helper: insert dataframe into BigQuery
def insert_to_bq(df, table_name):
    table_id = f"{BQ_PROJECT}.{BQ_DATASET}.{table_name}"
    job = client.load_table_from_dataframe(df, table_id)
    job.result()

# Transform Practitioners
def transform_practitioner_roles(rows):
    records = []
    for r in rows:
        rid, resource = r[1], r[2]

        # Initialize columns
        specialty_text = resource.get("specialty", {})[0].get("text")
        #specialty_code = resource.get("specialty", {})[0].get("code",[{}][0].get("coding",[{}][0].get("code")))
        specialty_code = resource.get("specialty", [{}])[0].get("coding", [{}])[0].get("code")
        role_text = resource.get("code", [{}])[0].get("text")
        role_code = resource.get("code", [{}])[0].get("coding", [{}])[0].get("code")
        
        print(specialty_code)
        print(specialty_text)
        print(role_text)
        print(role_code)

        

        records.append({
            "practitioner_role_id": rid,
            "practitioner_npi": resource.get("practitioner", {}).get("identifier").get("value"),
            "organization_id": resource.get("organization", {}).get("identifier").get("value"),
            "specialty_code": specialty_code,
            "specialty_text": specialty_text,
            "role_text" : role_text, 
            "role_code": role_code, 
            #                 if resource.get("telecom") and resource["telecom"][0].get("system")=="email" else None,
            #"role_text": resource.get("telecom", [{}])[0].get("value") 
                              #if resource.get("telecom") and resource["telecom"][0].get("system")=="phone" else None,
            "load_timestamp": datetime.now(timezone.utc)
        })

    return pd.DataFrame(records)


# Main ETL loop
def etl_practitioner_roles():
    try:
        for batch in fetch_staged_data("practitioner_roles_fhir_raw"):
            df = transform_practitioner_roles(batch)
            if not df.empty:
                insert_to_bq(df, "practitioner_roles")
                print("***Inserted batch***")
    except Exception as e:
        print(f"Error in practitioner roles ETL: {e}")

if __name__ == "__main__":
    etl_practitioner_roles()


208D00000X
General Practice Physician
General Practice Physician
208D00000X
208D00000X
General Practice Physician
General Practice Physician
208D00000X
208D00000X
General Practice Physician
General Practice Physician
208D00000X
208D00000X
General Practice Physician
General Practice Physician
208D00000X
208D00000X
General Practice Physician
General Practice Physician
208D00000X
***Inserted batch***


In [ ]:
#---Observations ETL
import psycopg2
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from datetime import datetime, timezone
import json
import pprint
import dateutil.parser  # optional, for robust ISO parsing

# Path to your service account JSON key
key_path = "/Users/toniventura/keys/bq_key.json"

# Create credentials and BigQuery client
credentials = service_account.Credentials.from_service_account_file(key_path)
BQ_PROJECT = "fhir-synthea-data"
BQ_DATASET = "fhir_curated"
client = bigquery.Client(project=BQ_PROJECT, credentials=credentials)

# Postgres config
PG_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "fhir",
    "user": "toniventura",
    "password": "fhir_project"
}

def insert_to_bq(df, table_name):
    table_id = f"{BQ_PROJECT}.{BQ_DATASET}.{table_name}"
    job = client.load_table_from_dataframe(df, table_id)
    job.result()
    print(f"Loaded {job.output_rows} rows to {table_id}")



# Helper: fetch staged data
def fetch_staged_data(table, batch_size=10000):
    try:
        conn = psycopg2.connect(**PG_CONFIG)
        cur = conn.cursor()
        cur.execute(f"SELECT * FROM fhir_staging.{table} LIMIT 5;")
        while True:
            rows = cur.fetchmany(batch_size)
            if not rows:
                break
            yield rows
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Postgres connection or query failed: {e}")

# Transform Practitioners
def transform_observations(rows):
    records = []
    for r in rows:
        rid, resource = r[1], r[2]

        # Initialize columns
        codings  = resource.get("code", {}).get("coding", [])
        code_text = resource.get("code", {}).get("text")

        status = resource.get("status") if codings else None
        system = codings[0].get("system")if codings else None
        code = codings[0].get("code") if codings else None
        
        codings_struct = [
            {
                "system" : c.get("system"), 
                "code" : c.get("code"), 
                "display": c.get("display")
                
            }for c in codings
        ]

        value_numeric = None
        unit = None
        value_text = None
        value_codings = []

        if "valueQuantity" in resource:
            q = resource["valueQuantity"]
            value_numeric = q.get("value")
            unit = q.get("unit")
            # system/code available but usually redundant here

        elif "valueString" in resource:
            value_text = resource["valueString"]

        elif "valueCodeableConcept" in resource:
            cc = resource["valueCodeableConcept"]
            # Store text in value_text for quick querying
            value_text = cc.get("text")
            # Capture codings for full fidelity
            for c in cc.get("coding", []):
                value_codings.append({
                "system": c.get("system"),
                "code": c.get("code"),
                "display": c.get("display")
             })

        elif "valueDateTime" in resource:
            value_text = resource["valueDateTime"]  # or a dedicated column

        elif "valuePeriod" in resource:
            value_text = json.dumps(resource["valuePeriod"])  # or expand into start/end columns

        patient_id_ref = resource.get("subject", {}).get("reference")
        patient_id = patient_id_ref.split(":")[-1]

        encounter_id_ref = resource.get("encounter",{}).get("reference")
        encounter_id = encounter_id_ref.split(":")[-1]

        effective_date_str = resource.get("effectiveDateTime")
        effective_date = None

        if effective_date_str:
            effective_datetime = dateutil.parser.isoparse(effective_date_str)
        
        records.append({
            "observation_id": rid,
            "status" : status,
            "obs_code": code,
            "system": system,
            "obs_code_text": code_text,
            "codings": codings_struct,
            "value_numeric": value_numeric, 
            "value_text" : value_text,
            "unit" : unit,
            "value_codings" : value_codings, 
            "patient_id" : patient_id,
            "encounter_id" : encounter_id,
            "effective_datetime": effective_date, #not inserting
            "load_timestamp": datetime.now(timezone.utc) 
        })

    return pd.DataFrame(records)


# Main ETL loop
def etl_observations():
    try:
        for batch in fetch_staged_data("observations_fhir_raw"):
            df = transform_observations(batch)
            if not df.empty:
                insert_to_bq(df, "observations")
                print("***Inserted batch***")
    except Exception as e:
        print(f"Error in observations ETL: {e}")

if __name__ == "__main__":
    etl_observations()


Loaded 5 rows to fhir-synthea-data.fhir_curated.observations
***Inserted batch***


In [21]:
#---Conditions ETL
import psycopg2
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from datetime import datetime, timezone
import json
import pprint
import dateutil.parser  # optional, for robust ISO parsing

# Path to your service account JSON key
key_path = "/Users/toniventura/keys/bq_key.json"

# Create credentials and BigQuery client
credentials = service_account.Credentials.from_service_account_file(key_path)
BQ_PROJECT = "fhir-synthea-data"
BQ_DATASET = "fhir_curated"
client = bigquery.Client(project=BQ_PROJECT, credentials=credentials)

# Postgres config
PG_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "fhir",
    "user": "toniventura",
    "password": "fhir_project"
}

def insert_to_bq(df, table_name):
    table_id = f"{BQ_PROJECT}.{BQ_DATASET}.{table_name}"
    job = client.load_table_from_dataframe(df, table_id)
    job.result()
    print(f"Loaded {job.output_rows} rows to {table_id}")



# Helper: fetch staged data
def fetch_staged_data(table, batch_size=10000):
    try:
        conn = psycopg2.connect(**PG_CONFIG)
        cur = conn.cursor()
        cur.execute(f"SELECT * FROM fhir_staging.{table} LIMIT 5;")
        while True:
            rows = cur.fetchmany(batch_size)
            if not rows:
                break
            yield rows
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Postgres connection or query failed: {e}")

# Transform Practitioners
def transform_conditions(rows):
    records = []
    for r in rows:
        rid, resource = r[1], r[2]

        # Initialize columns
        codings  = resource.get("code", {}).get("coding", [])
        code_text = resource.get("code", {}).get("text")

        status = resource.get("status") if codings else None
        system = codings[0].get("system")if codings else None
        code = codings[0].get("code") if codings else None
        
        codings_struct = [
            {
                "system" : c.get("system"), 
                "code" : c.get("code"), 
                "display": c.get("display")
                
            }for c in codings
        ]

        category = None
        category_text = None

        if resource.get("category"):
            first_category = resource["category"][0]["coding"][0]  # first category -> first coding
            category = first_category.get("code")
            category_text = first_category.get("display")
        else:
            category = None
            category_text = None

        # Nested array for full fidelity
        category_codings = []
        for cat in resource.get("category", []):
            for coding in cat.get("coding", []):
                category_codings.append({
                "system": coding.get("system"),
                "code": coding.get("code"),
                "display": coding.get("display")
             })

    
        patient_id_ref = resource.get("subject", {}).get("reference")
        patient_id = patient_id_ref.split(":")[-1]

        encounter_id_ref = resource.get("encounter",{}).get("reference")
        encounter_id = encounter_id_ref.split(":")[-1]

        onset_date_str = resource.get("effectiveDateTime")
        onset_date_time = None

        if onset_date_str:
            onset_date_time = dateutil.parser.isoparse(onset_date_str)
        
        records.append({
            "condition_id": rid,
            "clinical_status" : status,
            "code": code,
            "code_system": system,
            "code_text": code_text,
            "codings": codings_struct,
            "category_code": category,
            "category": category_text,
            "category_codings": category_codings,
            "patient_id" : patient_id,
            "encounter_id" : encounter_id,
            "onset_date": onset_date_time, #not inserting
            "load_timestamp": datetime.now(timezone.utc) 
        })

    return pd.DataFrame(records)


# Main ETL loop
def etl_conditions():
    try:
        for batch in fetch_staged_data("conditions_fhir_raw"):
            df = transform_conditions(batch)
            if not df.empty:
                insert_to_bq(df, "conditions")
                print("***Inserted batch***")
    except Exception as e:
        print(f"Error in conditions ETL: {e}")

if __name__ == "__main__":
    etl_conditions()


Loaded 5 rows to fhir-synthea-data.fhir_curated.conditions
***Inserted batch***


In [110]:
#---Claims ETL
import psycopg2
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from datetime import datetime, timezone
import json
import pprint
import dateutil.parser  # optional, for robust ISO parsing

# Path to your service account JSON key
key_path = "/Users/toniventura/keys/bq_key.json"

# Create credentials and BigQuery client
credentials = service_account.Credentials.from_service_account_file(key_path)
BQ_PROJECT = "fhir-synthea-data"
BQ_DATASET = "fhir_curated"
client = bigquery.Client(project=BQ_PROJECT, credentials=credentials)

# Postgres config
PG_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "fhir",
    "user": "toniventura",
    "password": "fhir_project"
}

def insert_to_bq(df, table_name):
    table_id = f"{BQ_PROJECT}.{BQ_DATASET}.{table_name}"
    job = client.load_table_from_dataframe(df, table_id)
    job.result()
    print(f"Loaded {job.output_rows} rows to {table_id}")



# Helper: fetch staged data
def fetch_staged_data(table, batch_size=10000):
    try:
        conn = psycopg2.connect(**PG_CONFIG)
        cur = conn.cursor()
        cur.execute(f"SELECT * FROM fhir_staging.{table} LIMIT 5;")
        while True:
            rows = cur.fetchmany(batch_size)
            if not rows:
                break
            yield rows
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Postgres connection or query failed: {e}")

# Transform Practitioners
def transform_claims(rows):
    records = []
    for r in rows:
        rid, resource = r[1], r[2]

        keys= resource.keys()

        print(f"keys: {keys}")

        use = resource.get("use")
        print(f"use: {use}")

        status = resource.get("status")
        print(f"status: {status}")

        patient_id = resource.get("patient", {}).get("reference").split(":")[-1]
        print(f"patient_id: {patient_id}")
        type = "None"
        type_info = resource.get("type", {})
        if isinstance (type_info, dict):
            print("dictionary")
            codings = type_info.get("coding")
            if codings:
                type = codings[0].get("code")
                print(f"Dictionary....type: {type}")
        
        elif isinstance(type_info, list) and type_info:
            first = type_info[0]
            if isinstance(first, dict):
                codings = first.get("coding")
                if codings:
                    type_info = codings[0].get("code")
                    print(f"type_info...type_info")
        else:
            type = None
        print(f"type_info: {type}")

        total_value = resource.get("total",{}).get("value")
        print(total_value)
        total_currency = resource.get("total", {}).get("currency")
        print(total_currency)

        billable_start = None
        billable_start = pd.to_datetime(resource.get("billablePeriod", {}).get("start"), utc=True)
        print(billable_start)
        billable_end = None
        billable_end = pd.to_datetime(resource.get("billablePeriod", {}).get("end"), utc=True)
        print(billable_end)

        date_created = pd.to_datetime(resource.get("created"), utc=True)
        print(f"date_created: {date_created}")

        billing_provider = []

        provider_obj = resource.get("provider", {})
        if provider_obj:  # only add if provider exists
            provider_reference = provider_obj.get("reference", "")
            provider_reference_id = provider_reference.split("|")[-1] if "|" in provider_reference else provider_reference
            provider_reference_type = provider_reference.split("?")[0] if "?" in provider_reference else None
            provider_display = provider_obj.get("display")

        '''billing_provider.append({
            "provider_reference_id": provider_reference_id,
            "provider_reference_type": provider_reference_type,
            "provider_display": provider_display
        })'''

        # ensure it is always a list, even if empty

        priority_codings= resource.get("priority", {}).get("coding",[]) 
        priority_code = priority_codings[0].get("code")
        print(f"priority_code: {priority_code}")

        facility_json = resource.get("facility")
        print(f"facility_obj: {facility_json}")
        #print(f"facility_json: {facility_json}")
        facility = []
        facility_reference = "None"
        facility_id = "None"
        facility_display = "None"
        
        if facility_json:
            facility_reference_complete = facility_json.get("reference")
            facility_display = facility_json.get("display")

        #print(f"facility: {facility_reference_complete}")
            if facility_reference_complete:
                facility_reference = facility_reference_complete.split("|")[0]
                print(f"facility_reference: {facility_reference}")
            
                facility_id = facility_reference_complete.split("|")[-1]
                print(f"facility_id: {facility_id}")
            else:
                facility_reference = "None"
                facility_id = "None"
        facility.append({
            "facility_reference": facility_reference,
            #"facility_reference": facility_json.get("reference"),
            "facility_id": facility_id,
            "facility_display" : facility_display  
        })

        insurances = resource.get("insurance",[])
        all_insurances = []

        for insurance in insurances:
            insurance_seq = insurance.get("sequence")
            print(f"Insurance seq: {insurance_seq}")
            insurance_focal =insurance.get("focal")
            print(f"Focal: {insurance_focal}")
            insurance_coverage = insurance.get("coverage", {}).get("display")
            print(f"Coverage {insurance_coverage}")

            all_insurances.append({
                "sequence": insurance_seq,
                "focal": insurance_focal,
                "coverage": insurance.get("coverage", {}).get("display")
            })
        
        
        items = resource.get("items", [{}])
        all_items = []

        for item in items:
            print("Items")
            sequence = "None"
            sequence = item.get("sequence")
            print(f"Item sequence: {sequence}")
            item_type = "None"
            diagnosis_seq = "None"
            diagnosis_seq = item.get("diagnosisSequence")
            information_seq = "None"
            information_seq = item.get("informationSequence")
            procedure_seq = "None"
            procedure_seq = item.get("procedureSequence")
            productOrService = item.get("productOrService")
            system = "None"
            code = "None"
            display = "None"
            if(productOrService):
                coding = productOrService.get("coding")
                if coding: 
                    system = coding[0].get("system")
                    print(f"system: {system}")
                    code = coding[0].get("code")
                    print(f"code: {code}")
                    display = coding[0].get("display")
                    print(f"display: {display}")
            service_period = "None"
            service_period = item.get("servicePeriod",{})
            print(f"service_period: {service_period}")
            start = "None"
            end = "None"
            if(service_period):
                start = service_period.get("start")
                end = service_period.get("end")
                
            print(f"start: {start}")
            print(f"end: {end}")

            if diagnosis_seq:
                item_type = "diagnosis sequence"  
            if information_seq:
                item_type = "information sequence"
            if procedure_seq:
                item_type = "procedure sequence"  
            print(f"item_type: {item_type}")

            location = []

            location_coding = item.get("locationCodeableConcept",[])
            location_system = "None"
            location_code = "None"
            location_display = "None"

            if location_coding:
                location_system = location_coding[0].get("system")
                location_code = location_coding[0].get("code")
                location_display = location_coding[0].get("display")
            
            '''print(f"location_system: {location_system}")
            print(f"location_code: {location_code}")
            print(f"location_display: {location_display}")'''

            location.append({
                "facility_id": facility_id,
                "system": location_system,
                "code": location_code,
                "display": location_display
            })

            print(f"-------------------location: {location}")

            encounter = None
            encounter_array = item.get("encounter",[])
            if encounter_array: 
                encounter = encounter_array[0].get("reference").split(":")[-1]
            print(f"encounter:{encounter}")

            net_value = 0
            net_currency = "USD"
            net = item.get("net",{})
            if net:
                net_value = net.get("value")
            print(f"net_value: {net_value}")
            if net:
                net_currency = net.get("currency")
            print(f"net_currency: {net_currency}")

            item_text =item.get("text")
            print(f"item_text: {item_text}")

            all_items.append({
                "sequence": sequence,
                "item_type": item_type,
                "system": system,
                "code": code,
                "display": display,
                "service_start":start,
                "service_end": end,
                "net_value": net_value,
                "net_currency": net_currency,
                "location": location,
                "encounter": encounter,
                "start_period": start,
                "end_period": end,
                "item_text": item_text
            })

        # Initialize columns
        #diagnosis_sequence = None
        #diagnosis_id = None
        #all_diagnosis = [{"sequence": None, "diagnosis": None}]
        '''all_diagnosis = []
        diagnoses = resource.get("diagnosis",[])
        print(f"diagnoses {diagnoses}")

        

        for diagnosis in diagnoses:
            print("--------------There are diagnoses.")
            diagnosis_sequence = diagnosis.get("sequence")
            diagnosis_id = diagnosis.get("diagnosisReference").get("reference").split(":")[-1] #conditions
            print(f"diagnosis: {diagnosis_id}")
            print(f"sequence: {diagnosis_sequence}")
            all_diagnosis.append({
                "sequence": diagnosis_sequence,
                "diagnosis": diagnosis_id   
            })
        
        print(f"all_diagnosis: {all_diagnosis}")'''

        diagnoses = resource.get("diagnosis", []) or []
        all_diagnosis = []

        if isinstance(diagnoses, list):
            for diagnosis in diagnoses:
                diag_ref = diagnosis.get("diagnosisReference", {}).get("reference")
                diagnosis_id = diag_ref.split(":")[-1] if diag_ref else None
                all_diagnosis.append({
                    "sequence": diagnosis.get("sequence"),
                    "diagnosis": diagnosis_id
                })

        # enforce consistent type: always a list (not None, not scalar)
        if not all_diagnosis:
            all_diagnosis = []

        records.append({
            "claim_id": rid,
            "status" : status,
            "use": use,
            "status": status,
            "patient_id": patient_id,
            "claim_type_info": type,
            "total_value": total_value,
            "total_currency": total_currency,
            "billable_start": billable_start,
            "billable_end" : billable_end,
            "created": date_created,
            #"billing_provider": [
                #{
            "provider_reference_id": provider_reference_id,
            "provider_reference_type": provider_reference_type,
            "provider_display": provider_display,
                #}
            #] if provider_reference_id else [],
            "priority_code": priority_code,
            "date_created": date_created,
            "facility": [
                {
                    "facility_reference": facility_reference,
                    "facility_id": facility_id,
                    "facility_display" : facility_display 
                }
            ] if facility_json else [],
            "all_insurances": all_insurances,
            "diagnoses": all_diagnosis,
            "items":all_items,
            "load_timestamp": datetime.now(timezone.utc) 
        })

    return pd.DataFrame(records)


# Main ETL loop
def etl_claims():
    try:
        for batch in fetch_staged_data("claims_fhir_raw"):
            df = transform_claims(batch)
            if not df.empty:
                insert_to_bq(df, "claims")
                print("***Inserted batch***")
    except Exception as e:
        print(f"Error in claims ETL: {e}")

if __name__ == "__main__":
    etl_claims()


keys: dict_keys(['id', 'use', 'item', 'type', 'total', 'status', 'created', 'patient', 'priority', 'provider', 'insurance', 'prescription', 'resourceType', 'billablePeriod'])
use: claim
status: active
patient_id: 9cfbd2b4-5eff-91db-1a93-777a9b86738f
dictionary
Dictionary....type: pharmacy
type_info: pharmacy
550.36
USD
2004-11-26 13:41:41+00:00
2004-11-26 13:56:41+00:00
date_created: 2004-11-26 13:56:41+00:00
priority_code: normal
facility_obj: None
Insurance seq: 1
Focal: True
Coverage NO_INSURANCE
Items
Item sequence: None
service_period: {}
start: None
end: None
item_type: None
-------------------location: [{'facility_id': 'None', 'system': 'None', 'code': 'None', 'display': 'None'}]
encounter:None
net_value: 0
net_currency: USD
item_text: None
keys: dict_keys(['id', 'use', 'item', 'type', 'total', 'status', 'created', 'patient', 'facility', 'priority', 'provider', 'diagnosis', 'insurance', 'resourceType', 'billablePeriod'])
use: claim
status: active
patient_id: 9cfbd2b4-5eff-91db-1

Error converting Pandas column with name: "diagnoses" and datatype: "object" to an appropriate pyarrow datatype: Array, ListArray, or StructArray


Error in claims ETL: Error converting Pandas column with name: "diagnoses" and datatype: "object" to an appropriate pyarrow datatype: Array, ListArray, or StructArray


In [130]:
#---Claims ETL
import psycopg2
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from datetime import datetime, timezone
import json
import pprint
import dateutil.parser  # optional, for robust ISO parsing
from google.cloud import bigquery

# Path to your service account JSON key
key_path = "/Users/toniventura/keys/bq_key.json"

# Create credentials and BigQuery client
credentials = service_account.Credentials.from_service_account_file(key_path)
BQ_PROJECT = "fhir-synthea-data"
BQ_DATASET = "fhir_curated"
client = bigquery.Client(project=BQ_PROJECT, credentials=credentials)

# Postgres config
PG_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "fhir",
    "user": "toniventura",
    "password": "fhir_project"
}

def insert_to_bq(df, table_name):
    table_id = f"{BQ_PROJECT}.{BQ_DATASET}.{table_name}"
    job = client.load_table_from_dataframe(df, table_id)
    job.result()
    print(f"Loaded {job.output_rows} rows to {table_id}")



# Helper: fetch staged data
def fetch_staged_data(table, batch_size=10000):
    try:
        conn = psycopg2.connect(**PG_CONFIG)
        cur = conn.cursor()
        cur.execute(f"SELECT * FROM fhir_staging.{table} LIMIT 5;")
        while True:
            rows = cur.fetchmany(batch_size)
            if not rows:
                break
            yield rows
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Postgres connection or query failed: {e}")

def safe_list_of_structs(input_list, required_keys):
    """Ensure a list of dicts is PyArrow-safe."""
    if not input_list or not isinstance(input_list, list):
        return [{k: None for k in required_keys}]
    cleaned = []
    for item in input_list:
        if not isinstance(item, dict):
            continue
        cleaned.append({k: item.get(k, None) for k in required_keys})
    return cleaned if cleaned else [{k: None for k in required_keys}]

def normalize_list_of_dicts(lst, keys):
    """Ensure lst is a list of dicts with all keys present, filling missing keys with None."""
    if not lst or not isinstance(lst, list):
        return []
    normalized = []
    for d in lst:
        if not isinstance(d, dict):
            continue
        normalized.append({k: d.get(k, None) for k in keys})
    return normalized

def transform_claims_bq(rows):
    records = []

    for r in rows:
        rid, resource = r[1], r[2]

        # --- Basic fields ---
        use = resource.get("use")
        status = resource.get("status")
        patient_ref = resource.get("patient", {}).get("reference")
        patient_id = patient_ref.split(":")[-1] if patient_ref else None

        # Claim type
        type_info = resource.get("type")
        claim_type = None
        if isinstance(type_info, dict):
            coding = type_info.get("coding")
            if coding:
                claim_type = coding[0].get("code")
        elif isinstance(type_info, list) and type_info:
            coding = type_info[0].get("coding")
            if coding:
                claim_type = coding[0].get("code")

        # --- Totals ---
        total_value = resource.get("total", {}).get("value")
        total_currency = resource.get("total", {}).get("currency")

        # --- Dates ---
        billable_start = resource.get("billablePeriod", {}).get("start")
        billable_end = resource.get("billablePeriod", {}).get("end")
        created = resource.get("created")

        # --- Provider ---
        provider_obj = resource.get("provider") or {}
        provider_reference = provider_obj.get("reference")
        provider_reference_id = provider_reference.split("|")[-1] if provider_reference else None
        provider_reference_type = provider_reference.split("?")[0] if provider_reference and "?" in provider_reference else None
        provider_display = provider_obj.get("display")

        '''billing_provider = safe_list_of_structs(
            [{"provider_reference_id": provider_id,
              "provider_reference_type": provider_type,
              "provider_display": provider_display}],
            ["provider_reference_id", "provider_reference_type", "provider_display"]
        )'''

        # --- Facility ---
        facility_json = resource.get("facility") or {}
        facility_reference_complete = facility_json.get("reference")
        facility_reference = facility_reference_complete.split("|")[0] if facility_reference_complete else None
        facility_id = facility_reference_complete.split("|")[-1] if facility_reference_complete else None
        facility_display = facility_json.get("display")

        facility = []
        if facility_json:
            facility.append({
                "facility_reference": facility_reference,
                "facility_id": facility_id,
                "facility_display": facility_display
            })

        # --- Priority ---
        priority_codings = resource.get("priority", {}).get("coding", [])
        priority_code = priority_codings[0].get("code") if priority_codings else None

        # --- Insurance ---
        insurances = resource.get("insurance", [])
        all_insurances = []
        for insurance in insurances:
            all_insurances.append({
                "sequence": insurance.get("sequence"),
                "focal": insurance.get("focal"),
                "coverage": insurance.get("coverage", {}).get("display")
            })

        ## --- Items ---
        items = resource.get("items", [])
        all_items = []
        for item in items:
            product = item.get("productOrService", {})
            coding = product.get("coding", [{}])[0] if product else {}
            location_coding = item.get("locationCodeableConcept", [{}])[0] if item.get("locationCodeableConcept") else {}

            encounter_array = item.get("encounter", [])
            encounter_ref = encounter_array[0].get("reference").split(":")[-1] if encounter_array else None

            net = item.get("net", {})
            all_items.append({
                "sequence": item.get("sequence"),
                "item_type": "diagnosis sequence" if item.get("diagnosisSequence") else (
                             "information sequence" if item.get("informationSequence") else (
                             "procedure sequence" if item.get("procedureSequence") else None)),
                "system": coding.get("system"),
                "code": coding.get("code"),
                "display": coding.get("display"),
                "service_start": item.get("servicePeriod", {}).get("start"),
                "service_end": item.get("servicePeriod", {}).get("end"),
                "net_value": net.get("value"),
                "net_currency": net.get("currency"),
                "location": [{
                    "facility_id": facility_id,
                    "system": location_coding.get("system"),
                    "code": location_coding.get("code"),
                    "display": location_coding.get("display")
                }] if location_coding else [],
                "encounter": encounter_ref,
                "item_text": item.get("text")
            })

         # --- Diagnoses ---
        diagnoses = resource.get("diagnosis", []) or []
        all_diagnosis = []
        for diag in diagnoses:
            diag_ref = diag.get("diagnosisReference", {}).get("reference")
            diag_id = diag_ref.split(":")[-1] if diag_ref else None
            all_diagnosis.append({
                "sequence": diag.get("sequence"),
                "diagnosis": diag_id
            })

        # --- Append record ---
        records.append({
            "claim_id": rid,
            "status": status,
            "use": use,
            "patient_id": patient_id,
            "claim_type_info": claim_type,
            "total_value": total_value,
            "total_currency": total_currency,
            "billable_start": billable_start,
            "billable_end": billable_end,
            "created": created,
            "provider_reference_id": provider_reference_id,
            "provider_reference_type": provider_reference_type,
            "provider_display": provider_display,
            "priority_code": priority_code,
            "facility": facility,
            "all_insurances": all_insurances,
            "diagnoses": all_diagnosis,
            "items": all_items,
            "load_timestamp": datetime.now(timezone.utc).isoformat()
        })
    return records

# Main ETL loop
def etl_claims():
    try:
        for batch in fetch_staged_data("claims_fhir_raw"):
            '''df = transform_claims(batch)
            if not df.empty:
                insert_to_bq(df, "claims")
                print("***Inserted batch***")'''
            

            table_id = "fhir-synthea-data.fhir_curated.claims"

        # Define schema (nested fields)
        job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("claim_id", "STRING", mode="REQUIRED"),
            bigquery.SchemaField("status", "STRING"),
            bigquery.SchemaField("use", "STRING"),
            bigquery.SchemaField("patient_id", "STRING"),
            bigquery.SchemaField("claim_type_info", "STRING"),
            bigquery.SchemaField("total_value", "FLOAT"),
            bigquery.SchemaField("total_currency", "STRING"),
            bigquery.SchemaField("billable_start", "TIMESTAMP"),
            bigquery.SchemaField("billable_end", "TIMESTAMP"),
            bigquery.SchemaField("created", "TIMESTAMP"),
            bigquery.SchemaField("provider_reference_id", "STRING"),
            bigquery.SchemaField("provider_reference_type", "STRING"),
            bigquery.SchemaField("provider_display", "STRING"),
            bigquery.SchemaField("priority_code", "STRING"),
            bigquery.SchemaField("facility", "RECORD", mode="REPEATED", fields=[
            bigquery.SchemaField("facility_reference", "STRING"),
            bigquery.SchemaField("facility_id", "STRING"),
            bigquery.SchemaField("facility_display", "STRING"),
        ]),
        bigquery.SchemaField("all_insurances", "RECORD", mode="REPEATED", fields=[
            bigquery.SchemaField("sequence", "INTEGER"),
            bigquery.SchemaField("focal", "BOOLEAN"),
            bigquery.SchemaField("coverage", "STRING"),
        ]),
        bigquery.SchemaField("diagnoses","RECORD", mode="REPEATED", fields=[
            bigquery.SchemaField("sequence", "INTEGER"),
            bigquery.SchemaField("diagnosis", "STRING"),
        ]),
        bigquery.SchemaField("items", "RECORD", mode="REPEATED", fields=[
            bigquery.SchemaField("sequence", "INTEGER"),
            bigquery.SchemaField("item_type", "STRING"),
            bigquery.SchemaField("system", "STRING"),
            bigquery.SchemaField("code", "STRING"),
            bigquery.SchemaField("display", "STRING"),
            bigquery.SchemaField("service_start", "STRING"),
            bigquery.SchemaField("service_end", "STRING"),
            bigquery.SchemaField("net_value", "FLOAT"),
            bigquery.SchemaField("net_currency", "STRING"),
            bigquery.SchemaField("location", "RECORD", mode="REPEATED", fields=[
                bigquery.SchemaField("facility_id", "STRING"),
                bigquery.SchemaField("system", "STRING"),
                bigquery.SchemaField("code", "STRING"),
                bigquery.SchemaField("display", "STRING"),
            ]),
            bigquery.SchemaField("encounter", "STRING"),
            bigquery.SchemaField("item_text", "STRING"),
        ]),
        bigquery.SchemaField("load_timestamp", "TIMESTAMP", mode="REQUIRED"),
        ],
        )

        # Load data
        records = transform_claims_bq(batch)
        job = client.load_table_from_json(records, table_id, job_config=job_config)
        job.result()
        print(f"Loaded {len(records)} rows to {table_id}")

    except Exception as e:
        print(f"Error in claims ETL: {e}")

if __name__ == "__main__":
    etl_claims()


Loaded 5 rows to fhir-synthea-data.fhir_curated.claims
